In [88]:
import os
import sys
import pandas as pd
import numpy as np
import time
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'src'))
from database import DB
from utils import dot_wma


In [81]:
pd.set_option('mode.chained_assignment', None)

In [51]:
def getEntriesByTeam(df, team, league, year=None, season=None, seasonal = False):
    # Get partial data frame
    if( seasonal ):
        new_df = df[ (df['league_name'] == league) & (df['year'] == year) & (df['season'] == season) & (df['corresponding_team'] == team) ]
    else:
        new_df = df[(df['league_name'] == league) & (df['corresponding_team'] == team)]
    return new_df.sort_values(by=['date']).reset_index(drop=True)

In [47]:
db = DB()
db.initialise()

df = db.get_table('view_set_match_info_concat')  #df.shape = (35692, 57)

In [71]:
temp_df = df[df['wdl'].notnull()]

In [72]:
temp_df.head()

,league_name,year,season,date,set_number,team_1,team_2,corresponding_team,tiebreaker,match_round,...,team_minion_kills,team_monster_kills,team_goldat10,team_goldat15,team_golddiffat10,team_golddiffat15,team_csat10,team_csat15,team_csdiffat10,team_csdiffat15
0,BL,2020,spring,2020-01-21,1,Sector One,Aethra,Sector One,0,1,...,640.0,167.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BL,2020,spring,2020-01-21,1,Brussels Guardians,KV Mechelen,Brussels Guardians,0,1,...,911.0,249.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BL,2020,spring,2020-01-21,1,Brussels Guardians,RSCA,Brussels Guardians,0,1,...,651.0,139.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BL,2020,spring,2020-01-21,1,Sector One,Timeout Esports,Sector One,0,1,...,751.0,191.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BL,2020,spring,2020-01-21,1,Sector One,Timeout Esports,Timeout Esports,0,1,...,719.0,167.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [93]:
concat_df = pd.DataFrame()
rolling_window = 3
for league in temp_df['league_name'].unique():
    temp_league = temp_df[temp_df['league_name']==league]
    for team in temp_league['corresponding_team'].unique():
        temp_team = temp_league[temp_league['corresponding_team']==team].sort_values(by=['date']).reset_index(drop=True)
        temp_team['cum_win_rate'] = temp_team['wdl'].apply(lambda x: 1 if x == 'W' else 0).expanding(1).mean().shift(1)
        for year in temp_team['year'].unique():
            temp_year = temp_team[temp_team['year']==year]
            for season in temp_year['season'].unique():
                temp_season = temp_year[temp_year['season']==season]
                temp_season['win_weighted_sum'] = temp_season['wdl'].apply(lambda x: 1 if x == 'W' else 0).rolling(rolling_window).apply(lambda x: dot_wma(x, rolling_window)).shift(1)
                concat_df = pd.concat([concat_df, temp_season]).reset_index(drop=True)
        

In [82]:
concat_df

,league_name,year,season,date,set_number,team_1,team_2,corresponding_team,tiebreaker,match_round,...,team_goldat10,team_goldat15,team_golddiffat10,team_golddiffat15,team_csat10,team_csat15,team_csdiffat10,team_csdiffat15,cumWinRate,recentWinRate
0,BL,2020,spring,2020-01-21,1,Sector One,Aethra,Sector One,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BL,2020,spring,2020-01-21,1,Sector One,Timeout Esports,Sector One,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN
2,BL,2020,spring,2020-01-28,1,RSCA,Sector One,Sector One,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN
3,BL,2020,spring,2020-02-04,1,KV Mechelen,Sector One,Sector One,0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000
4,BL,2020,spring,2020-02-11,1,Brussels Guardians,Sector One,Sector One,0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33704,VCS,2020,summer,2020-08-08,2,OverPower Esports,EVOS Esports,OverPower Esports,0,8,...,15772.0,26003.0,19.0,-401.0,282.0,456.0,-30.0,-25.0,0.214286,0.000000
33705,VCS,2020,summer,2020-08-14,1,Team Flash,OverPower Esports,OverPower Esports,0,9,...,15642.0,24980.0,526.0,-299.0,274.0,444.0,-26.0,-50.0,0.206897,0.000000
33706,VCS,2020,summer,2020-08-14,2,Team Flash,OverPower Esports,OverPower Esports,0,9,...,15419.0,24184.0,-1469.0,-3598.0,268.0,452.0,-37.0,-58.0,0.200000,0.000000
33707,VCS,2020,summer,2020-08-16,1,Percent Esports,OverPower Esports,OverPower Esports,0,9,...,16286.0,24449.0,-306.0,-592.0,316.0,516.0,-4.0,-7.0,0.193548,0.000000


In [94]:
concat_df

,league_name,year,season,date,set_number,team_1,team_2,corresponding_team,tiebreaker,match_round,...,team_goldat10,team_goldat15,team_golddiffat10,team_golddiffat15,team_csat10,team_csat15,team_csdiffat10,team_csdiffat15,cumWinRate,WWS
0,BL,2020,spring,2020-01-21,1,Sector One,Aethra,Sector One,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BL,2020,spring,2020-01-21,1,Sector One,Timeout Esports,Sector One,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN
2,BL,2020,spring,2020-01-28,1,RSCA,Sector One,Sector One,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN
3,BL,2020,spring,2020-02-04,1,KV Mechelen,Sector One,Sector One,0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.955335
4,BL,2020,spring,2020-02-11,1,Brussels Guardians,Sector One,Sector One,0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.955335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33704,VCS,2020,summer,2020-08-08,2,OverPower Esports,EVOS Esports,OverPower Esports,0,8,...,15772.0,26003.0,19.0,-401.0,282.0,456.0,-30.0,-25.0,0.214286,0.000000
33705,VCS,2020,summer,2020-08-14,1,Team Flash,OverPower Esports,OverPower Esports,0,9,...,15642.0,24980.0,526.0,-299.0,274.0,444.0,-26.0,-50.0,0.206897,0.000000
33706,VCS,2020,summer,2020-08-14,2,Team Flash,OverPower Esports,OverPower Esports,0,9,...,15419.0,24184.0,-1469.0,-3598.0,268.0,452.0,-37.0,-58.0,0.200000,0.000000
33707,VCS,2020,summer,2020-08-16,1,Percent Esports,OverPower Esports,OverPower Esports,0,9,...,16286.0,24449.0,-306.0,-592.0,316.0,516.0,-4.0,-7.0,0.193548,0.000000


In [ ]:
def getRelativeEntriesByTeam( index,  row ):
    prev_record = df[ df.index < index ]
    opponent_team = row['team_2'] if (row['corresponding_team'] == row['team_1']) else row['team_1']
    entry_df = prev_record[
                (prev_record['corresponding_team'] == row['corresponding_team'])
                & ((prev_record['team_1'] == opponent_team) | (prev_record['team_2'] == opponent_team))]
    if( len(entry_df) == 0 ):
        return
    print( entry_df[['league_name', 'year', 'season', 'team_1', 'team_2', 'corresponding_team', 'wdl']] )

In [100]:
aa = pd.DataFrame(data=[[1,2,3],[2,3,4],[3,4,5]], columns=list('abc'))
col_to_use = list('abc')
aa[[x+'_1' for x in col_to_use]] = aa.apply(lambda x: x+1)

In [101]:
aa

,a,b,c,a_1,b_1,c_1
0,1,2,3,2,3,4
1,2,3,4,3,4,5
2,3,4,5,4,5,6


In [99]:
col_to_use

['a', 'b', 'c']